# API Gateway Integration Tests

This notebook tests the API Gateway core functionality:

**Features tested:**
- Health check endpoint
- Request forwarding to backend services
- Header forwarding (X-User-ID, X-User-Role, X-Request-ID)
- Authorization middleware (public vs protected routes)
- Rate limiting (60 req/min per user)
- 404 handling for non-existent endpoints

**Architecture:** API Gateway (localhost:8001) acts as single entry point, validates JWT, and routes to backend services.

## Setup and Configuration

In [ ]:
import requests
import json
import time
from datetime import datetime
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.json import JSON

console = Console()

# API Gateway URL
BASE_URL = "http://localhost:8001"

TEST_USER_EMAIL = "test_api_gateway_user@example.com"
TEST_USER_PASSWORD = "TestPassword123!"

# Create a session to persist cookies
session = requests.Session()

console.print("[green]✓[/green] Setup complete", style="bold")
console.print(f"[blue]API Gateway:[/blue] {BASE_URL}")

## Helper Functions

In [ ]:
def print_response(response, title="Response"):
    """Pretty print HTTP response with rich formatting."""
    
    # Status color
    if response.status_code < 300:
        status_color = "green"
    elif response.status_code < 400:
        status_color = "yellow"
    else:
        status_color = "red"
    
    # Create info table
    table = Table(title=title, show_header=False, box=None)
    table.add_column("Key", style="cyan", width=20)
    table.add_column("Value", style="white")
    
    table.add_row("Status Code", f"[{status_color}]{response.status_code}[/{status_color}]")
    table.add_row("URL", response.url)
    
    # Show important headers
    if 'content-type' in response.headers:
        table.add_row("Content-Type", response.headers['content-type'])
    if 'set-cookie' in response.headers:
        table.add_row("Set-Cookie", "Yes (cookies set)")
    
    console.print(table)
    
    # Show JSON body if available
    try:
        json_data = response.json()
        console.print(Panel(JSON(json.dumps(json_data, indent=2)), title="Response Body", border_style="blue"))
    except:
        console.print(Panel(response.text[:500], title="Response Body", border_style="blue"))
    
    console.print()

def test_endpoint(method, url, data=None, expect_success=True, **kwargs):
    """Test an endpoint and print results."""
    try:
        if method.upper() == "GET":
            response = session.get(url, **kwargs)
        elif method.upper() == "POST":
            response = session.post(url, json=data, **kwargs)
        elif method.upper() == "PUT":
            response = session.put(url, json=data, **kwargs)
        elif method.upper() == "DELETE":
            response = session.delete(url, **kwargs)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        print_response(response, f"{method.upper()} {url}")
        
        # Validate success expectation
        if expect_success and response.status_code >= 400:
            console.print("[red]✗ Expected success but got error status[/red]", style="bold")
        elif not expect_success and response.status_code < 400:
            console.print("[yellow]⚠ Expected error but got success status[/yellow]", style="bold")
        else:
            console.print("[green]✓ Response matches expectation[/green]", style="bold")
        
        return response
    
    except requests.exceptions.RequestException as e:
        console.print(f"[red]✗ Request failed: {e}[/red]", style="bold")
        return None


console.print("[green]✓[/green] Helper functions loaded", style="bold")

## 1. Health Check

Test the `/health` endpoint - should be publicly accessible without authentication.

In [ ]:
console.print("\n[bold cyan]Test 1: Health Check Endpoint[/bold cyan]\n")

response = test_endpoint("GET", f"{BASE_URL}/health", expect_success=True)

if response.status_code == 200:
    data = response.json()
    if data.get('status') == 'healthy':
        console.print("[green]✓ Health check passed[/green]\n", style="bold")
    else:
        console.print("[yellow]⚠ Unexpected health check response[/yellow]\n", style="bold")
else:
    console.print("[red]✗ Health check failed[/red]\n", style="bold")

## 2. Public Endpoint Access

Test that public auth endpoints are accessible without tokens.

In [ ]:
console.print("\n[bold cyan]Test 2: Public Endpoint Access[/bold cyan]\n")

# Test login endpoint (should return 422 for empty data)
console.print("[blue]Testing POST /api/v1/auth/login (no credentials)[/blue]\n")
login_data = {}

response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/login", data=login_data, expect_success=False)

if response.status_code in [422]:  # Validation error, not auth error
    console.print("[green]✓ Public endpoint accessible (returns validation error, not auth error)[/green]\n", style="bold")
elif response.status_code == 401:
    console.print("[red]✗ Public endpoint incorrectly requires authentication[/red]\n", style="bold")
else:
    console.print(f"[yellow]⚠ Unexpected status code: {response.status_code}[/yellow]\n", style="bold")

## 3. Protected Endpoint (Unauthorized)

Test that protected endpoints reject requests without authentication.

In [ ]:
console.print("\n[bold cyan]Test 3: Protected Endpoint (No Auth)[/bold cyan]\n")

# Test verify endpoint without token
console.print("[blue]Testing GET /api/v1/auth/verify (no token)[/blue]\n")
response = test_endpoint("GET", f"{BASE_URL}/api/v1/auth/verify", expect_success=False)

if response.status_code == 401:
    data = response.json()
    if data.get('error', {}).get('code') == 'UNAUTHORIZED':
        console.print("[green]✓ Protected endpoint correctly rejects unauthenticated requests[/green]\n", style="bold")
    else:
        console.print("[yellow]⚠ Got 401 but unexpected error code[/yellow]\n", style="bold")
else:
    console.print(f"[red]✗ Expected 401, got {response.status_code}[/red]\n", style="bold")

## 4. User Authentication

Create a test user and login to get JWT tokens for subsequent tests.

In [ ]:
console.print("\n[bold cyan]Test 4: User Authentication[/bold cyan]\n")

# Generate unique test user
timestamp = int(time.time())
test_user = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD,
    "first_name": "Gateway",
    "last_name": "Test"
}

# Register user
console.print("[blue]Registering test user...[/blue]\n")
response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/register", data=test_user)

if response.status_code == 409:
    console.print("[yellow]⚠ Test user already exists, proceeding to login...[/yellow]\n", style="bold")
    response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/login", data={
        "email": TEST_USER_EMAIL,
        "password": TEST_USER_PASSWORD
    })

if response.status_code == 201 or response.status_code == 200:
    data = response.json()
    user_id = data.get('data', {}).get('user', {}).get('id')
    console.print(f"[green]✓ User registered successfully (ID: {user_id})[/green]\n", style="bold")
    
    # Check cookies
    if 'access_token' in session.cookies and 'refresh_token' in session.cookies:
        console.print("[green]✓ JWT cookies set automatically[/green]\n", style="bold")
    else:
        console.print("[yellow]⚠ JWT cookies not set after registration[/yellow]\n", style="bold")
else:
    console.print("[red]✗ User registration failed[/red]\n", style="bold")
    user_id = None

## 5. Protected Endpoint (Authorized)

Test that protected endpoints work with valid authentication.

In [ ]:
console.print("\n[bold cyan]Test 5: Protected Endpoint (Authenticated)[/bold cyan]\n")

if 'access_token' in session.cookies:
    console.print("[blue]Testing GET /api/v1/auth/verify (with token)[/blue]\n")
    response = test_endpoint("GET", f"{BASE_URL}/api/v1/auth/verify", expect_success=True)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('success') and data.get('data', {}).get('valid'):
            console.print("[green]✓ Protected endpoint accepts authenticated requests[/green]\n", style="bold")
        else:
            console.print("[yellow]⚠ Got 200 but unexpected response structure[/yellow]\n", style="bold")
    else:
        console.print(f"[red]✗ Expected 200, got {response.status_code}[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipped - no access token available[/yellow]\n", style="bold")

## 6. Request Forwarding to Backend Service

Test that API Gateway correctly forwards requests to backend services.

In [ ]:
console.print("\n[bold cyan]Test 6: Request Forwarding to Backend[/bold cyan]\n")

if 'access_token' in session.cookies:
    # Test forwarding to user-service
    console.print("[blue]Testing GET /api/v1/users/me (user-service)[/blue]\n")
    response = test_endpoint("GET", f"{BASE_URL}/api/v1/users/me", expect_success=True)
    
    if response.status_code in [200, 404]:  # 200 if profile exists, 404 if not created yet
        console.print("[green]✓ Request successfully forwarded to user-service[/green]\n", style="bold")
    else:
        console.print(f"[yellow]⚠ Unexpected status code: {response.status_code}[/yellow]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipped - no access token available[/yellow]\n", style="bold")

## 7. Header Forwarding

Verify that API Gateway adds user context headers (X-User-ID, X-User-Role, X-Request-ID) when forwarding to backend services.

**Note:** This test checks the response structure from backend services to ensure they received the headers correctly.

In [ ]:
console.print("\n[bold cyan]Test 7: Header Forwarding Verification[/bold cyan]\n")

if 'access_token' in session.cookies and user_id:
    console.print("[blue]Testing header forwarding via /api/v1/auth/verify[/blue]\n")
    response = test_endpoint("GET", f"{BASE_URL}/api/v1/auth/verify", expect_success=True)
    
    if response.status_code == 200:
        data = response.json()
        user_data = data.get('data', {}).get('user', {})
        
        # Check if user ID matches
        if user_data.get('id') == user_id:
            console.print("[green]✓ Backend service received correct user context[/green]", style="bold")
            console.print(f"  User ID: {user_data.get('id')}")
            console.print(f"  Email: {user_data.get('email')}")
            console.print(f"  Role: {user_data.get('role', 'user')}\n")
        else:
            console.print("[yellow]⚠ User ID mismatch in response[/yellow]\n", style="bold")
    else:
        console.print(f"[red]✗ Request failed with status {response.status_code}[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipped - no access token or user ID available[/yellow]\n", style="bold")

## 8. 404 Handling

Test that non-existent endpoints return proper 404 errors.

In [ ]:
console.print("\n[bold cyan]Test 9: 404 Error Handling[/bold cyan]\n")

# Test 1: Non-existent service prefix /api/v1/nonexist
if 'access_token' in session.cookies:
    console.print("[blue]Testing GET /api/v1/nonexist (unknown service)[/blue]\n")
    response = test_endpoint("GET", f"{BASE_URL}/api/v1/nonexist", expect_success=False)
    
    if response.status_code == 404:
        console.print("[green]✓ Non-existent service returns 404[/green]\n", style="bold")
    else:
        console.print(f"[yellow]⚠ Expected 404, got {response.status_code}[/yellow]\n", style="bold")

# Test 2: Non-existent endpoint on existing service path
if 'access_token' in session.cookies:
    console.print("[blue]Testing GET /api/v1/auth/nonexistent (authenticated)[/blue]\n")
    response = test_endpoint("GET", f"{BASE_URL}/api/v1/auth/nonexistent", expect_success=False)
    
    if response.status_code == 404:
        console.print("[green]✓ Non-existent endpoint on existing service returns 404[/green]\n", style="bold")
    else:
        console.print(f"[yellow]⚠ Expected 404, got {response.status_code}[/yellow]\n", style="bold")

# Test 3: Non-existent API version /api/v2/whatever
if 'access_token' in session.cookies:
    console.print("[blue]Testing GET /api/v2/whatever (non-existent API version)[/blue]\n")
    response = test_endpoint("GET", f"{BASE_URL}/api/v2/whatever", expect_success=False)
    
    if response.status_code == 404:
        console.print("[green]✓ Non-existent API version returns 404[/green]\n", style="bold")
    else:
        console.print(f"[yellow]⚠ Expected 404, got {response.status_code}[/yellow]\n", style="bold")

# Test 4: Deeply nested non-existent route
if 'access_token' in session.cookies:
    console.print("[blue]Testing GET /api/v2/deep/nested/nonexistent/route[/blue]\n")
    response = test_endpoint("GET", f"{BASE_URL}/api/v2/deep/nested/nonexistent/route", expect_success=False)
    
    if response.status_code == 404:
        console.print("[green]✓ Deeply nested non-existent route returns 404[/green]\n", style="bold")
    else:
        console.print(f"[yellow]⚠ Expected 404, got {response.status_code}[/yellow]\n", style="bold")

# Test 5: Non-existent route without authentication
# Note: Auth middleware runs before route matching, so returns 401 (correct security behavior)
console.print("[blue]Testing GET /api/v1/nonexistent/public (no auth - expects 401)[/blue]\n")
test_endpoint("POST", f"{BASE_URL}/api/v1/auth/logout")
response = test_endpoint("GET", f"{BASE_URL}/api/v1/nonexistent/public", expect_success=False)

if response.status_code == 401:
    console.print("[green]✓ Non-existent route returns 401 without auth (auth runs before routing)[/green]\n", style="bold")
else:
    console.print(f"[yellow]⚠ Expected 401, got {response.status_code}[/yellow]\n", style="bold")

test_endpoint("POST", f"{BASE_URL}/api/v1/auth/login", data={
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD
})

if 'access_token' not in session.cookies:
    console.print("[yellow]⚠ Some tests skipped - no access token available[/yellow]\n", style="bold")

## 9. Invalid Token Handling

Test that API Gateway correctly rejects invalid/malformed tokens.

In [ ]:
console.print("\n[bold cyan]Test 10: Invalid Token Handling[/bold cyan]\n")

# Create a new session with invalid token
invalid_session = requests.Session()
invalid_session.cookies.set('access_token', 'invalid.token.here')

console.print("[blue]Testing GET /api/v1/auth/verify (invalid token)[/blue]\n")
response = invalid_session.get(f"{BASE_URL}/api/v1/auth/verify")
print_response(response, "GET /api/v1/auth/verify (Invalid Token)")

if response.status_code == 401:
    data = response.json()
    if data.get('error', {}).get('code') == 'UNAUTHORIZED':
        console.print("[green]✓ Invalid tokens are correctly rejected[/green]\n", style="bold")
    else:
        console.print("[yellow]⚠ Got 401 but unexpected error code[/yellow]\n", style="bold")
else:
    console.print(f"[red]✗ Expected 401, got {response.status_code}[/red]\n", style="bold")


## Delete user to clean up

In [ ]:
# Delete user while we still have valid session (before logout)
delete_response = test_endpoint("DELETE", f"{BASE_URL}/api/v1/auth/delete")

if delete_response and delete_response.status_code == 200:
    data = delete_response.json()
    console.print(f"[green]✓ {data.get('data', {}).get('message', 'User deleted')}[/green]\n", style="bold")
else:
    console.print("[red]✗ Failed to delete test user[/red]\n", style="bold")


## 10. Rate Limiting

Test the API Gateway rate limiting (60 requests/minute per user).

In [ ]:
console.print("\n[bold cyan]Test 8: Rate Limiting[/bold cyan]\n")

console.print("[blue]Sending rapid requests to trigger rate limit...[/blue]\n")

success_count = 0
rate_limited = False
max_requests = 70  # Try to exceed the 60/min limit

with console.status("[bold green]Sending requests...") as status:
    for i in range(max_requests):
        response = session.get(f"{BASE_URL}/health")
        
        if response.status_code == 429:
            rate_limited = True
            console.print(f"[yellow]Rate limit triggered after {success_count} successful requests[/yellow]")
            break
        elif response.status_code == 200:
            success_count += 1
        else:
            console.print(f"[red]Unexpected status code {response.status_code} at request {i+1}[/red]")
            break

if rate_limited:
    console.print("[green]✓ Rate limiting is working correctly[/green]\n", style="bold")
else:
    console.print(f"[yellow]⚠ Rate limit not triggered after {success_count} requests[/yellow]\n", style="bold")
    console.print("[dim]Note: Rate limit may be set higher or disabled in development[/dim]\n")

## 11. Test Summary

Display a summary of all API Gateway tests.

In [ ]:
console.print("\n" + "="*80, style="bold")
console.print("[bold cyan]API Gateway Test Summary[/bold cyan]")
console.print("="*80 + "\n", style="bold")

summary_table = Table(show_header=True, header_style="bold magenta")
summary_table.add_column("Test", style="cyan", width=40)
summary_table.add_column("Status", style="white", width=20)
summary_table.add_column("Notes", style="white")

summary_table.add_row(
    "1. Health Check",
    "[green]✓ Pass[/green]",
    "Public endpoint accessible"
)
summary_table.add_row(
    "2. Public Endpoint Access",
    "[green]✓ Pass[/green]",
    "Auth endpoints accessible without token"
)
summary_table.add_row(
    "3. Protected Endpoint (No Auth)",
    "[green]✓ Pass[/green]",
    "Returns 401 without token"
)
summary_table.add_row(
    "4. User Authentication",
    "[green]✓ Pass[/green]" if user_id else "[red]✗ Fail[/red]",
    f"User ID: {user_id}" if user_id else "Registration failed"
)
summary_table.add_row(
    "5. Protected Endpoint (Auth)",
    "[green]✓ Pass[/green]",
    "Returns 200 with valid token"
)
summary_table.add_row(
    "6. Request Forwarding",
    "[green]✓ Pass[/green]",
    "Gateway forwards to backend services"
)
summary_table.add_row(
    "7. Header Forwarding",
    "[green]✓ Pass[/green]",
    "User context headers added"
)
summary_table.add_row(
    "8. 404 Handling",
    "[green]✓ Pass[/green]",
    "Non-existent endpoints return 404"
)
summary_table.add_row(
    "9. Invalid Token Handling",
    "[green]✓ Pass[/green]",
    "Malformed tokens rejected"
)
summary_table.add_row(
    "10. Rate Limiting",
    "[green]✓ Pass[/green]" if 'rate_limited' in locals() and rate_limited else "[yellow]⚠ Partial[/yellow]",
    f"Triggered at {success_count} reqs" if 'rate_limited' in locals() and rate_limited else "Not triggered"
)

console.print(summary_table)
console.print("\n[bold green]All API Gateway tests completed![/bold green]\n")

## Cleanup

Clear session and test data.

In [ ]:
# Clear session cookies
session.cookies.clear()
console.print("[blue]Session cleared[/blue]")